#**Milestone 2**

In [ ]:
# It is recommended to upgrade the statsmodels library. 
# Uncomment the below code to upgrade statsmodels
!pip install statsmodels --upgrade

In [ ]:
import pandas as pd
import warnings
import itertools
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#to ignore warnings
import warnings
import itertools
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('MER_T12_06.xlsx')
df.head()

In [ ]:
#conversion of "YYYYMM" columnn into standard datetime format & making it as index
# We are using errors=’coerce’. It will replace all non-numeric values with NaN.

dateparse = lambda x: pd.to_datetime(x, format='%Y%m', errors = 'coerce')
df = pd.read_excel('MER_T12_06.xlsx', parse_dates=['YYYYMM'], index_col='YYYYMM', date_parser=dateparse) 
df.head()

**The arguments can be explained as:**

- **parse_dates:** This is a key to identify the date time column. Example, the column name is ‘YYYYMM’.
- **index_col:** This is a key that forces pandas to use the date time column as index.
- **date_parser:** Converts an input string into datetime variable.

- Let us first identify and **drop the non datetimeindex** rows. First, let's convert the index to datetime, coerce errors, and filter NaT

In [ ]:
ts = df[pd.Series(pd.to_datetime(df.index, errors='coerce')).notnull().values]
ts.head()
ts.describe().T

**Observations**
1. The observations have reduced to 4707 after filtering on NaT
2. There are 9 unique categories in MSN and Description columns
3. The 'Value' coulmn has missing values with a high frequency of 384. The rows with these missing values should be eliminated

In [ ]:
#convert the emision value into numeric value
nat=pd.DataFrame(pd.to_numeric(ts['Value'],errors='coerce')).convert_dtypes()
ts['Value']=nat['Value']

In [ ]:
#Drop the missing value using dropna(inplace = True)
ts.dropna(inplace = True)
ts.describe().T

In [ ]:
ts.dtypes

### **Natural gas based CO2 emission forecasting**

For developing the time series model and forecasting, you are expected to use the natural gas CO2 emission from the electrical power generation. We need to slice this data:

In [ ]:
###Slice the data to get the monthly total CO2 emissions of Natural Gas Electric Power Sector
natural=ts[ts['MSN']=='NNEIEUS']
natural= pd.DataFrame(natural).drop(['Description','MSN'],axis=1)

In [ ]:
 #Check 1st few rows of data
natural.head()

###**Split the dataset**

In [ ]:
# Split the data into train and test
# using first 34 years data as training data
train_data = natural.loc['1973-01-01':'2007-11-01']

# using the last 9 years data as test data
test_data = natural.loc['2007-12-01':'2016-07-01']
print(train_data)
print(test_data)

###**Test the Stationarity**

In [ ]:
#Import the required package

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

###**Test the stationarity through Visualization**

In [ ]:
# creating a subplot space
fig, ax = plt.subplots(figsize=(16, 6))

# plotting train data
train_data.plot(ax=ax)

# plotting test data
test_data.plot(ax=ax)

# adding the legends in sequential order
plt.legend(['train data', 'test data'])

# showing the time which divides the original data into train and test
plt.axvline(x='2007-12-01', color='black', linestyle='--')

# showing the plot
plt.show()

In [ ]:
# Calculate the rolling mean and standard deviation for a window of 12 observations
rolmean=train_data.rolling(window=12).mean()
rolstd=train_data.rolling(window=12).std()

# Visualize the rolling mean and standard deviation
plt.figure(figsize=(16,8))
actual = plt.plot(train_data, color='cyan', label='Actual Series')
rollingmean = plt.plot(rolmean, color='red', label='Rolling Mean') 
rollingstd = plt.plot(rolstd, color='green', label='Rolling Std. Dev.')
plt.title('Rolling Mean & Standard Deviation of the Series')
plt.legend()
plt.show()

#### **Observations and Insights: ____**
1. Series has upward trend, it is not stationary

### **Test the stationarity using the Augmented Dickey-Fuller Test**

Use the **Augmented Dickey-Fuller (ADF) Test** to verify if the series is stationary or not. The null and alternate hypotheses for the ADF Test are defined as:

**- Null hypothesis:** The Time Series is non-stationary


**- Alternative hypothesis:** The Time Series is stationary

In [ ]:
#Define a function to use adfuller test
def adfuller(train_data):
  #Importing adfuller using statsmodels
  from statsmodels.tsa.stattools import adfuller
  print('Dickey-Fuller Test: ')
  adftest = adfuller(train_data['Value'])
  adfoutput = pd.Series(adftest[0:4], index=['Test Statistic','p-value','Lags Used','No. of Observations'])
  for key,value in adftest[4].items():
    adfoutput['Critical Value (%s)'%key] = value
  print(adfoutput)
adfuller(train_data)

- **Observations and Insights**
 
Observations:

1. From the above test, we can see that the p-value = 0.995 i.e. > 0.05 (For 95% confidence intervals) therefore, we fail to reject the null hypothesis.
2. Hence, we can confirm that the series is non-stationary.

###**Transformation of the dataset into a stationary one**

**We can use some of the following methods to convert a non-stationary series into a stationary one:**


1. Log Transformation
2. Differencing the series (lagged series)

We take the average of ‘k’ consecutive values depending on the frequency of time series (in this capstone 12 months). 

Here, we will take the average over the past 1 year.

In [ ]:
 # Visualize the rolling mean and standard deviation after using log transformation
plt.figure(figsize=(16,8))
df_log = np.log(train_data) #Reduced variance
MAvg = df_log.rolling(window=12).mean()
MStd = df_log.rolling(window=12).std()
plt.plot(df_log)
plt.plot(MAvg, color='r', label = 'Moving Average')
plt.plot(MStd, color='g', label = 'Standard Deviation')
plt.legend()
plt.show()

**Observations and Insights: _____**
- Since **we can still see the upward trend in the series**, we can conclude that **the series is still non-stationary.** 
- However, the standard deviation is almost constant which implies that **now the series has constant variance.**

**Visualize the rolling mean and rolling standard deviation of the shifted series (df_shift) and check the stationarity by calling the adfuller() function. Also, write your observations on the same.**

In [ ]:
plt.figure(figsize=(16,8))
df_shift = df_log - df_log.shift(periods = 1,fill_value=0)
MAvg_shift = df_shift.rolling(window=12).mean()
MStd_shift = df_shift.rolling(window=12).std()
plt.plot(df_shift, color='c')
#plt.plot(MAvg_shift, color='red', label = 'Moving Average')
#plt.plot(MStd_shift, color='green', label = 'Standard Deviation')
plt.legend()
plt.show()

#Dropping the null values that we get after applying differencing method
df_shift = df_shift.dropna()

**Observations and Insights:___**
1.Since we can no longer see an upward trend, the series seems to be alomonst constant (stationary)

2. The standard deviation also seems to be almost constant

Lets verify using Augmented Dickey-Fuller (ADF) Test

In [ ]:
adfuller(df_shift)

**Observations and Insights: _____**
1. From the above test, we can see that the p-value = 5.447548e-14 i.e. < 0.05 (For 95% confidence intervals) therefore, we can reject the null hypothesis.
2. Hence, we can confirm that the series is now stationary.



# Obtaining stationarity through differencing##

In [ ]:
from statsmodels.tsa.stattools import adfuller
# implementing ADF test on the original time series data
result = adfuller(train_data['Value'])

fig, ax = plt.subplots(figsize=(16, 6))
train_data.plot(ax=ax)
plt.show()

# printing the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
# taking seasonal differencing of the timeseries
train_data_stationary = train_data.diff(periods=12).dropna()

# implementing ADF test on the first order differenced time series data
result = adfuller(train_data_stationary['Value'])

fig, ax = plt.subplots(figsize=(16, 6))
train_data_stationary.plot(ax=ax)
plt.show()

# printing the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
# taking 1st order differencing of the timeseries
train_data_stationary = train_data.diff().dropna()

# implementing ADF test on the first order differenced time series data
result = adfuller(train_data_stationary['Value'])

fig, ax = plt.subplots(figsize=(16, 6))
train_data_stationary.plot(ax=ax)
plt.show()

# printing the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])

**Observations and Insights: _____**
1. p-value=0.00062<0.05. Stationarity is obtained by 1st order differencing. Therefore d=1 in ARIMA modelling

Let's decompose the time series to check its different components.

### **Elimination of trend and seasonality: Decomposition**

In [ ]:
#Importing the seasonal_decompose function to decompose the time series

from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(train_data)

trend = decomp.trend
seasonal = decomp.seasonal
residual = decomp.resid

plt.figure(figsize=(15,10))
plt.subplot(411)
plt.plot(train_data, label='Actual', marker='.')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend, label='Trend', marker='.')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal, label='Seasonality', marker='.')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(residual, label='Residuals', marker='.')
plt.legend(loc='upper left')
plt.tight_layout()

**Observations and Insights: ____**
- We can see that there are significant **trend, seasonality and residuals components** in the series
- The plot for seasonality shows that **Natural gas based CO2 emissions spike in July and August.**

**Now let's move on to the model building section. First, we will plot the `ACF` and `PACF` plots to get the values of p and q i.e. order of AR and MA models to be used.**

**Plot the auto-correlation function and partial auto-correlation function to get p and q values for AR, MA, ARMA, and ARIMA models**

### **Find optimal parameters (P, Q) and build the AR, MA, ARMA & ARIMA models**

**Plot the ACF and PACF charts and find the optimal parameters**

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

plt.figure(figsize = (16,8))
plot_acf(df_shift, lags = 24) 
plt.show() 
plot_pacf(df_shift, lags = 24) 
plt.show()

**Observations and Insights: _____**

**Observations:**
- From the above PACF plot we can see that **the lag** at which the plot extends beyond the statistically significant boundary for the first time is **lag 1.** 
- This indicates that an **AR Model of lag 1 (p=1)** should be sufficient to fit the data.
- Similarly, from the ACF plot, we can infer that **q=1.**
- The ACF and PACF also capture the seasonality in the data




###**AR Model**

Order p is the lag value after which the PACF plot crosses the upper confidence interval for the first time. These p lags will act as our features while forecasting the AR time series.

Fit and predict the shifted series with the AR Model and calculate the RMSE. Also, visualize the time series and write your observations.

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error

plt.figure(figsize=(16,8))
model_AR = AutoReg(df_shift.astype(float), lags=1)#Use number of lags as 1 and apply AutoReg function on df_shift series
results_AR = model_AR.fit() #fit the model
plt.plot(df_shift)
predict = results_AR.predict() #predict the series 
predict = predict.fillna(0) #Converting NaN values to 0
plt.plot(predict, color='red')
plt.title('AR Model - RMSE: %.4f'% mean_squared_error(predict,df_shift['Value'], squared=False))  #Calculating rmse
plt.show()

**Observations & Insights: _____**
**Observations:________________________**
1. The RMSE value is 0.2. It is very less. Therefore,this is a good model if we only want to use the AR component while modeling.
2.The time series appears stationary

**Let's check the AIC value** of the model

In [ ]:
results_AR.aic

Now, let's build MA, ARMA, and ARIMA models as well, and see if we can get a better model 

###**MA Model**

####**Think about it:**

- Do we really have to find AR & I value other than 0 to forecast on the MA based model?

Order q of the MA process is obtained from the ACF plot, this is the lag after which ACF crosses the upper confidence interval for the first time.

Fit and predict the shifted series with the MA Model and calculate the RMSE. Also, visualize the time series and write your observations.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

plt.figure(figsize=(16,8))
model_MA =ARIMA(df_shift.astype(float), order=(0,0,1))#Using p=0, d=0, q=1 and apply ARIMA function on df_shift series
results_MA = model_MA.fit()#fit the model
plt.plot(df_shift)
predict1 = results_MA.predict() #predict the series 
predict1 = predict1.fillna(0) 
plt.plot(predict, color='red')
plt.title('MA Model - RMSE: %.4f'% mean_squared_error(results_MA.fittedvalues,df_shift['Value'], squared=False))
plt.show()

**Observations & Insights: _____**
**Observations:________________________**
1. THE RMSE is 0.2. Similar to AutoReg AR model. We need to check AIC value to determine if MA model is better than AR model for forecasting
2. The time series appears stationary

Let's check the AIC value of the model

In [ ]:
results_MA.aic

- **The MA model is giving higher AIC** when compared to the AR model, implying that **the AR model fits the training data better.** 

###**ARMA MODEL**

**We will be using the above AR lag(P) & MA lag(Q) as a paramter** and d=0 in ARIMA so that it will work as an ARMA model.

Fit and predict the shifted series with the ARMA Model and calculate the RMSE. Also, visualize the time series and write your observations.

In [ ]:
#Code here
plt.figure(figsize=(16,8))
model_ARMA =ARIMA(df_shift.astype(float), order=(1,0,1)) #Using p=1, d=0, q=1 and apply ARIMA function on df_shift series
results_ARMA =model_MA.fit() #fit the model
plt.plot(df_shift)
predict2 = results_MA.predict() #predict the series 
predict2 = predict2.fillna(0) 
plt.plot(predict, color='red')
plt.title('ARMA Model - RMSE: %.4f'% mean_squared_error(results_ARMA.fittedvalues,df_shift['Value'], squared=False))
plt.show()



**Observations & Insights: _____**
**Observations:**
1. THE RMSE is 0.0902. Similar to AutoReg AR model and MA(0,0,1) model
2.The time series appears stationary.


**Let's check the AIC value** of the model

**Check the AIC value of the model**

In [ ]:
#Code here
results_ARMA.aic

- **The AIC value of the ARMA model is more or less similar** to MA model 

**Let us try using the ARIMA Model.**

###**ARIMA MODEL**

**Fit and predict the shifted series with the ARIMA Model and calculate the RMSE. Also, visualize the time series and write your observations.**

In [ ]:
#Code here
from statsmodels.tsa.arima.model import ARIMA

plt.figure(figsize=(16,8))
model_ARIMA = ARIMA(df_log.astype(float), order=(1,1,1))#Using p=1, d=1, q=1 and apply ARIMA function on df_log series
results_ARIMA = model_ARIMA.fit()#fit the model
plt.plot(df_log)
predict3 = results_ARIMA.predict() #predict the series 
predict3 = predict3.fillna(0)
plt.plot(predict3, color='red')
plt.title('ARIMA Model - RMSE: %.4f'% mean_squared_error(results_ARIMA.fittedvalues,df_log['Value'], squared=False))
plt.show()

**Observations:________**
1. The RMSE value is 0.1965, lesser than previous models. The predicted values fit the data very well

**Let's check the AIC value** of the model

**Check the AIC value of the model**

In [ ]:
results_ARIMA.aic

From the above analysis, we can see that the ARIMA(1, 1, 1) is the best model as compared to others, as it has less RMSE as compared to all the other models. 

###**Inverse Transformation**

**Use the correct inverse transformation depending on the model chosen to get back the original values.**



**Apply an inverse transformation on the predictions of the chosen model**

In [ ]:
# Printing the fitted values
predictions=pd.Series(results_ARIMA.fittedvalues)
predictions

In [ ]:
#Third step - applying exponential transformation
predictions_ARIMA = np.exp(predictions)#use exponential function
predictions_ARIMA

**Plot the original vs predicted series**

In [ ]:
#Code here
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(train_data, color = 'c', label = 'Original Series')  #plot the original train series
plt.plot(predictions_ARIMA, color = 'r', label = 'Predicted Series')  #plot the predictions_ARIMA 
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

**Observations & Insights: _____**
 We can see that **the predicted series is very similar to the original series** i.e. The model is good at predicting values on the training data.
- Let us **forecast the closing prices for the next 24 months.**

###**Forecast the values for next 24 months and compare it with test data**

In [ ]:
#Add the code blocks based on the requirements
forecasted_ARIMA = results_ARIMA.forecast(steps=24)#forecast using the results_ARIMA for next 24 months. Keep steps=24
forecasted_ARIMA

In [ ]:
list1 = forecasted_ARIMA.tolist()
series1 = pd.Series(list1)
series1

In [ ]:
index = pd.date_range('2007-12-01','2009-12-01' , freq='1M')- pd.offsets.MonthBegin(1)
df1 = pd.DataFrame()
df1['forecasted'] = np.exp(series1)
df1.index = index
df1

In [ ]:
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(train_data, color = 'c', label = 'Original Series')
plt.plot(predictions_ARIMA, color = 'r', label = 'Prediction on Train data') #plot the predictions_ARIMA series
plt.plot(df1, label = 'Forecast', color='b')  #plot the forecasted_ARIMA series
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

**Observations:**
- **As observed earlier, most of the predicted values on the training data are very close to the actual values** 
- **The model does not capture seasonalities in the data**, therefore the forecasted values are not able to identify these seasonalities and therefore not close to the actual data

Let's test the RMSE of the transformed predictions and the original value on the training and testing data to check whether the model is giving a generalized performance or not.

In [ ]:
from sklearn.metrics import mean_squared_error
error =np.sqrt(mean_squared_error(predictions_ARIMA, train_data)) #calculate RMSE using the predictions_ARIMA and df_train 
error

In [ ]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(forecasted_ARIMA, test_data.iloc[0:24,:]))
error#calculate RMSE using the forecasted_ARIMA and df_test

####**Think about it:**
- Can we use other than RMSE measurement to check the performance of the model?

####**Think about it:**

 Can we use other forecasting methods such as SARIMA to improve our model performance?

- [A Gentle Introduction to SARIMA for Time Series Forecasting in Python](https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/)
- [Forecasting with Seasonal ARIMA in Python](https://www.datasciencecentral.com/profiles/blogs/tutorial-forecasting-with-seasonal-arima)

## **Proposed Approach**

####**Refined insights**:
- What are the most meaningful insights from the data relevant to the problem?
1. The CO2 emissions due to Natural gas have an increasing trend. The time series also consists of seasonalities (with peak emissions in the months of July, Sepetember) and residuals

####**Comparison of various techniques and their relative performance**:
- How do different techniques perform? Which one is performing relatively better? Is there scope to improve the performance further?
1. The ARIMA (1,1,1) fits better than all models. The RMSE value is lower than AR,MA and ARMA models. However, AIC values of MA and ARMA models are lower than all models
2. ARIMA model could not forecast CO2 emissions taking the seasonalities into account. THerfore, SARIMA would be better at identifying seasonalities in the forecasted values

####**Proposal for the final solution design**:
- What model do you propose to be adopted? Why is this the best solution to adopt?
1. I would obtain ACF and PACF plots of the stationary data (After turning non-stationary data to stationary)
2. I would obtain p and q values from the PACF and ACF plots respectively
3. I would use SARIMA model to fit the training data and forecast the next 24 months. This model should be able to identify seasonalities in the forecasted data

In [ ]:
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 3
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA: ')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
# Determing p,d,q combinations with AIC scores.
for param in pdq:
    for param_seasonal in seasonal_pdq:
        mod = sm.tsa.statespace.SARIMAX(df_log.astype(float),
                                        order=param,
                                        seasonal_order=param_seasonal,
                                        enforce_stationarity=False,enforce_invertibility=False)

        results = mod.fit(disp=0)

        print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df_log.astype(float),
                                order=(1, 1, 1),
                                seasonal_order=(1, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit(disp=0)

print(round(results.aic,2))
print(results.summary().tables[1])
results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Printing the fitted values
predictions=pd.Series(results.fittedvalues)
predictions

In [ ]:
#Third step - applying exponential transformation
predictions_SARIMA = np.exp(predictions)#use exponential function
predictions_SARIMA

In [ ]:
#Code here
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(train_data, color = 'c', label = 'Original Series')  #plot the original train series
plt.plot(predictions_SARIMA, color = 'r', label = 'Predicted Series')  #plot the predictions_ARIMA 
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

In [ ]:
#Add the code blocks based on the requirements
forecasted_SARIMA = results.forecast(steps=24)#forecast using the results_ARIMA for next 24 months. Keep steps=24
forecasted_SARIMA

In [ ]:
list2 = forecasted_SARIMA.tolist()
series2 = pd.Series(list2)
series2

In [ ]:
index = pd.date_range('2007-12-01','2009-12-01' , freq='1M')- pd.offsets.MonthBegin(1)
df2 = pd.DataFrame()
df2['forecasted'] = np.exp(series2)
df2.index = index
df2

In [ ]:
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(train_data, color = 'c', label = 'Original Series')
plt.plot(predictions_SARIMA, color = 'r', label = 'Prediction on Train data') #plot the predictions_ARIMA series
plt.plot(df2, label = 'Forecast', color='b')  #plot the forecasted_ARIMA series
plt.title('Actual vs Predicted')
plt.legend()
plt.show()